# Day 16 notebook

The objectives of this notebook are to practice

* Fitch's algorithm
* Estimating a tree based on parsimony
* Reconstructing ancestral sequences based on parsimony

## Modules used for this assignment

In [ ]:
import toytree # for working with trees
import fasta   # for reading alignments stored in FASTA files

## Important Python data structures used in this assignment
You should become familiar with how to use two data structures that will be important for implementing the algorithms in this activity.

### Sets
You have used the Python [set](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset) class before, but have perhaps not used the common set operators.  Here are some important ones for this assignment:

In [ ]:
A = {'C', 'G', 'T'}
B = {'A', 'C'}
print("The intersection of A and B is:", A & B)
print("       The union of A and B is:", A | B)

### Toytree trees
We have used the [toytree](https://toytree.readthedocs.io/) module for drawing trees, but there is also significant functionality provided by this module for traversing trees and accessing attributes of nodes within trees.  This functionality is contained within the `treenode` attribute of a toytree object.  A reference for all functionality of this object can be found in the [documentation for the ETE Toolkit Master Tree class](http://etetoolkit.org/docs/latest/reference/reference_tree.html).  Below I will highlight the methods and attributes that you will need in this activity.

First, here is a tree that we will use in a few examples of this functionality and a function for drawing rooted trees in a convenient way for this activity:

In [ ]:
def draw_tree_with_internal_labels(t):
    """Draws the given toytree tree with all nodes labeled to show their names"""
    t.draw(node_labels=t.get_node_values(feature="name", show_root=True, show_tips=True),
           tip_labels=False,
           use_edge_lengths=False,
           orient="down")

In [ ]:
test_newick = '((D,C1),(PB,(C2,PA)));'
test_tree = toytree.tree(test_newick)
draw_tree_with_internal_labels(test_tree)

The `treenode` attribute of a toytree tree represents the root node of the tree.  You can access the name of any given node using the `name` attribute of a node.

In [ ]:
print("The name of the root node is:", test_tree.treenode.name)

You can traverse all of the nodes within the subtree below a given node with the `traverse` method.  You can tell the traverse method the order in which you wish to traverse the nodes, which can either be "preorder", "postorder", or "levelorder".

In [ ]:
for order in ('preorder', 'postorder', 'levelorder'):
    print(order + ":", [node.name for node in test_tree.treenode.traverse(order)])

You can access the children and parent of a node via a tree node objects `children` and `up` attributes, repsectively.

In [ ]:
for node in test_tree.treenode.traverse('preorder'):
    print(node.name, 
          "Parent:", node.up.name if not node.is_root() else None, 
          "Children:", ", ".join(child.name for child in node.children))

Lastly, you can use the convenience methods `is_root` and `is_leaf` to check if a tree node is a root or leaf node, respectively.

In [ ]:
for node in test_tree.treenode.traverse('preorder'):
    print(node.name, 
          "is_root:", node.is_root(), 
          "is_leaf:", node.is_leaf())

## Data used for this assignment
We will again use the data from the case of possible HIV transmission from dentist to patient that we considered in the Day 15 notebook.  Recall that the CDC had performed DNA sequencing on a set of HIV samples and subsequent phylogenetic analyses to determine whether or not the molecular data provided evidence that the dentist had transmitted HIV to his patients.  If you read the paper ([Ou et al. Science, 1992](http://science.sciencemag.org/content/256/5060/1165)), you will find that the researchers actually used parsimony based methods to estimate trees.  In this activity we will redo some their analyses by implementing (unweighted) parsimony algorithms and running the algorithms on a multiple alignment of the V3 variable region of the HIV genome for these samples.

Below we will read in the multiple alignment that will be used as data for this activity.  The sequences from samples of HIV from the dentist (D), patient A (PA), patient B (PB), local control 1 (C1), and local control 2 (C2).

In [ ]:
v3_alignment_filename = "v3_alignment.fasta"
v3_aligned_sequences = fasta.read_sequences_from_fasta_file(v3_alignment_filename)
v3_sequence_names, v3_alignment = zip(*v3_aligned_sequences)

## PROBLEM 1: Fitch's Algorithm (stage 1) (1 POINT)

We will begin by implementing the first stage of Fitch's algorithm for computing the minimum cost of a tree for a single column of a multiple alignment, i.e., where each leaf node is assigned a single character.  Recall that the first stage of Fitch's algorithm traverses the tree in a post-order and computes the possible states at each node that minimize the cost of the tree below that node.  In addition, this stage computes the minimum cost of the tree, which can be determined by counting the number of times the set *union* operator is used.  Implement this algorithm in the function below.

In [ ]:
def fitch_score_and_min_cost_states(tree, leaf_states):
    """Runs the first stage of Fitch's algorithm for
       the given tree and character states as the leaves.
    
    Args:
        tree: a toytree tree.
        leaf_states: a dictionary mapping leaf names to characters.  
    Returns:
        A two-element tuple, where the first element is the minimum
        cost of the tree (minimum number of changes required to explain
        the leaf data) and second element is a dictionary mapping the
        node names to sets of possible states at the nodes (the R values
        in the algorithm)
    """
    R = {}
    num_changes = 0
    for node in tree.treenode.traverse("postorder"):
        if node.is_leaf():
            R[node.name] = {leaf_states[node.name]}
        else:
            ###
            ### YOUR CODE HERE
            ###
    return num_changes, R

In [ ]:
# tests for fitch_score_and_min_cost_states
test1_tree = toytree.tree('((D,C1),(PB,(C2,PA)));')
test1_leaf_states = {'PA': 'C', 'C2': 'A', 'C1': 'T', 'D': 'C', 'PB': 'C'}
test1_result = (2,
 {'D': {'C'},
  'C1': {'T'},
  'PB': {'C'},
  'C2': {'A'},
  'PA': {'C'},
  '5': {'A', 'C'},
  '6': {'C'},
  '7': {'C', 'T'},  
  '8': {'C'}})
assert fitch_score_and_min_cost_states(test1_tree, test1_leaf_states) == test1_result

test2_tree = toytree.tree('((D,C1),(PB,(C2,PA)));')
test2_leaf_states = {'PA': 'C', 'C2': 'C', 'C1': 'T', 'D': 'T', 'PB': 'C'}
test2_result = (1,
 {'D': {'T'},
  'C1': {'T'},
  'PB': {'C'},
  'C2': {'C'},
  'PA': {'C'},
  '5': {'C'},
  '6': {'C'},
  '7': {'T'},  
  '8': {'C', 'T'}})
assert fitch_score_and_min_cost_states(test2_tree, test2_leaf_states) == test2_result

test3_tree = toytree.tree('((D,C1),(PB,(C2,PA)));')
test3_leaf_states = {'PA': 'T', 'C2': 'T', 'C1': 'T', 'D': 'T', 'PB': 'T'}
test3_result = (0,
 {'D': {'T'},
  'C1': {'T'},
  'PB': {'T'},
  'C2': {'T'},
  'PA': {'T'},
  '5': {'T'},
  '6': {'T'},
  '7': {'T'},  
  '8': {'T'}})
assert fitch_score_and_min_cost_states(test3_tree, test3_leaf_states) == test3_result
print("SUCCESS: fitch_score_and_min_cost_states passed all tests")

### PROBLEM 2: Parsimony score for an entire alignment (1 POINT)
We will next use your `fitch_score_and_min_cost_states` function to compute the parsimony score for an entire alignment, which is simply the sum of the minimum costs for each column in the alignment.  Write a function `score_tree_parsimony` that takes as input a tree, an alignment, a list of the sequence names, and outputs the parsimony score.  You will likely find the function below, which returns a list of leaf state dictonaries, one leaf state dictionary per column of the alignment.

In [ ]:
def alignment_leaf_states_list(alignment, sequence_names):
    """Returns a list of dictionaries, where each dictionary corresponds to the leaf states
    for a column of the alignment."""
    return [dict(zip(sequence_names, column)) for column in zip(*alignment)]

In [ ]:
def score_tree_parsimony(tree, alignment, sequence_names):
    """Computes the parsimony score for a given tree and alignment.
    
    Args:
        tree: a toytree tree object
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        The parsimony score (a number)
    """
    ###
    ### YOUR CODE HERE
    ###


In [ ]:
# tests for score_tree_parsimony
test1_tree = toytree.tree('((D,C1),(PB,(C2,PA)));')
assert score_tree_parsimony(test1_tree, v3_alignment, v3_sequence_names) == 64
test2_tree = toytree.tree('((C1,C2),(PB,(D,PA)));')
assert score_tree_parsimony(test2_tree, v3_alignment, v3_sequence_names) == 58
print("SUCCESS: score_tree_parsimony passed all tests")

### Finding the most parsimonious trees for the HIV samples
Now we will use your parsimony functions to find the most parsimonious trees for the HIV sequence data.  To do this we will simply use a brute force method that computes the parsimony score for each possible rooted tree of five leaves.  Newick strings for all such trees with leaves labeled by the HIV sample names are provided in the file `all_five_leaf_rooted_trees.txt` and we will read these strings in below.

In [ ]:
all_possible_trees_filename = "all_five_leaf_rooted_trees.txt"
all_possible_trees = [line.strip() for line in open(all_possible_trees_filename)]

Next, here is a function that take runs your `score_tree_parsimony` on a list of newick string trees, and returns the minimum parsimony score along with the newick strings for the trees that obtain that score:

In [ ]:
def best_trees_parsimony(newick_tree_list, alignment, sequence_names):
    """Computes the minimum parsimony score and the trees that obtain that score, 
    given a list of candidate trees.
    
    Args:
        newick_tree_list: a list of newick string trees
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        A two-element tuple, with the first element giving the minimum parsimony score
        the second element being a list of newick string trees that obtain that score.
    """
    tree_scores = [score_tree_parsimony(toytree.tree(newick), alignment, sequence_names) 
                   for newick in newick_tree_list]
    min_score = min(tree_scores)
    return min_score, [newick for newick, score in zip(newick_tree_list, tree_scores) if score == min_score]

In [ ]:
v3_best_score, v3_best_trees = best_trees_parsimony(all_possible_trees, v3_alignment, v3_sequence_names)
print("V3 best score:", v3_best_score)
print("V3 best trees:", *v3_best_trees, sep="\n")

Many of trees are actually equivalent in their unrooted forms, and since the unweighted parsimony score is independent of the position of the root, it will make sense to consider just the distinct unrooted trees represented by this list.  Below is a function that takes as input a list of newick strings for rooted trees and returns a list of the distinct unrooted trees represented by those trees.

In [ ]:
def distinct_unrooted_trees(rooted_tree_newick_list):
    """Given a list of rooted trees in newick string format, returns a list of all distict
    unrooted trees corresponding to these trees."""
    distinct_trees = []
    tree_ids = set()
    for newick in rooted_tree_newick_list:
        unrooted_tree = toytree.tree(newick).unroot()
        unrooted_tree_id = unrooted_tree.treenode.get_topology_id()
        if unrooted_tree_id not in tree_ids:
            tree_ids.add(unrooted_tree_id)
            distinct_trees.append(unrooted_tree.write(tree_format=9))
    return distinct_trees

In [ ]:
v3_best_trees_unrooted = distinct_unrooted_trees(v3_best_trees)
print(*v3_best_trees_unrooted, sep="\n")

Some questions to think about with regard to these trees:
1. How well do these trees match up with the trees you constructed by UPGMA and neighbor joining in the Day 20 activity?
2. Are these trees supportive of the hypothesis that the dentist transmitted HIV to these two patients?

###
### Your thoughts here
###


### PROBLEM 3: Reconstructing ancestral sequences using parsmiony (1 POINT)
For the last problem, we will implement the second stage of Fitch's algorithm, which selects a single ancestral state for each internal node of a tree in a configuration that achieves the minimum cost.  Recall that the second stage of Fitch's algorithm traverses the tree in a pre-order and determines the state at each node that minimizes the cost, given the state selected at that node's parent.  This second stage takes uses the sets R computed in the first stage.

In [ ]:
def fitch_ancestral_states(tree, R):
    """Computes a configuration of states for the nodes of the tree that minimizes the cost of the tree.
    
    When visiting a node, if there are multiple states that give the same cost, 
    the lexicographically smaller state will be chosen.
    
    Args:
        tree: a toytree tree object
        R: the dictionary of possible ancestral states for each node (e.g., as computed by the
           fitch_score_and_min_cost_states function)
    Returns:
        A dictionary mapping node names to character states.
    """
    r = {} # a dictionary mapping node names to character states
    for node in tree.treenode.traverse("preorder"):
        if node.is_root():
            r[node.name] = sorted(R[node.name])[0] # use the lexicographically smallest element
        else:
            ###
            ### YOUR CODE HERE
            ###
    return r

In [ ]:
# tests for fitch_ancestral_states
test1_tree = toytree.tree('((D,C1),(PB,(C2,PA)));')
test1_R = {
    'C1': {'T'},
    'C2': {'A'},
    'D': {'C'},
    'PA': {'C'},
    'PB': {'C'},
    '5': {'A', 'C'},
    '6': {'C'},
    '7': {'C', 'T'},
    '8': {'C'}}
test1_states = {
    'C1': 'T', 
    'C2': 'A',
    'D': 'C',
    'PA': 'C',
    'PB': 'C',
    '5': 'C',
    '6': 'C',
    '7': 'C',
    '8': 'C'}
assert fitch_ancestral_states(test1_tree, test1_R) == test1_states

test2_tree = toytree.tree('((D,C1),(PB,(C2,PA)));')
test2_R = {
    'C1': {'T'},
    'C2': {'C'},
    'D': {'T'},
    'PA': {'C'},
    'PB': {'C'},
    '5': {'C'},
    '6': {'C'},
    '7': {'T'},
    '8': {'C', 'T'}}
test2_states = {
    'C1': 'T',
    'C2': 'C',
    'D': 'T',
    'PA': 'C',
    'PB': 'C',
    '5': 'C',
    '6': 'C',
    '7': 'T',
    '8': 'C'}
assert fitch_ancestral_states(test2_tree, test2_R) == test2_states

print("SUCCESS: fitch_ancestral_states passed all tests")

### Computing a putative ancestral sequence for the dentist's original strain of HIV
Finally, we can use your ancestral reconstruction algorithm to reconstruct the entire sequences at the internal nodes of the putative trees that we have reconstructed.  Below is a function that calls the `fitch_ancestral_states` function to reconstruct the ancestral characters at all columns in the multiple alignment.

In [ ]:
def compute_ancestral_sequences(tree, alignment, sequence_names):
    """Computes (reconstructed) sequences for all nodes in the given tree, given a multiple alignment.
    
    Args:
        tree: a toytree tree object
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        A dictionary mapping node names to strings (reconstructed sequences).
    """
    fitch_results = [fitch_score_and_min_cost_states(tree, leaf_states) 
                     for leaf_states in alignment_leaf_states_list(alignment, sequence_names)]
    column_scores, column_Rs = zip(*fitch_results)
    column_ancestral_states = [fitch_ancestral_states(tree, column_R) for column_R in column_Rs]
    node_names = [node.name for node in tree.treenode.traverse("postorder")]
    full_alignment = [''.join([states[name] for states in column_ancestral_states]) for name in node_names]
    return {name: sequence for name, sequence in zip(node_names, full_alignment)}

We can use this function to estimate what the sequence would be for the internal node that is the most recent common ancestor of the dentist and patient samples.  Assuming the hypothesis that the dentist trasmitted HIV to the patients, this internal node represents the sequence of the virus as it existed around the time that the transmission events occurred.  Below is the tree that we will assume in this case.

In [ ]:
possible_tree = toytree.tree("((D,(PA,PB)),(C1,C2));")
draw_tree_with_internal_labels(possible_tree)

And here is the reconstructed sequence.

In [ ]:
all_node_sequences = compute_ancestral_sequences(possible_tree, v3_alignment, v3_sequence_names)
ancestral_dentist_hiv_sequence = all_node_sequences["6"]
print(ancestral_dentist_hiv_sequence)